In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_gaussian_quantiles, make_classification
from sklearn.model_selection import train_test_split

# FairLabel: Correcting Bias in Labels

## Introduction

As ML models are becoming of more and more significance in taking various important decisions each day, algorithmic fairness is more crucial than ever. There is plenty of research on algorithmic fairness but most of it is concentrated on algorithmic bias - bias that is not present in the data but added purely by the algorithm. 

Sociatal bias and historical discrimination both plague the data we base our precious models on, which will therefore make biased predictions. To combat biased data we use preprocessing methods like preferential sampling and disperate impact removal which remove data biases but keep labels intact. It is impossible to manually re-label ground truth labels because of the sheer amount of time it would take and the lack of information about how the decisions were made. That is the purpose of FairLabel - to mitigate label bias.

## Metrics for Debiasing Task

In biased data we have a minority group and a majority group and we can see favourable treatment towards the majority group. A biased decision occurs when the decision maker, having the right qualification, makes a wrong classification purely based on which group the subject belongs to. Overall, the probability for a favourable outcome for a member of the minority class is lower:

$$P(y=1\vert p=minority) < P(y=1\vert p=majority)$$

What FairLabel will do is flip $(0\rightarrow 1)$ for every mistaken label in the minorty class and do the opposite - $(1\rightarrow 0)$, for the majority class. In the context of flipping, TPR (True Positive Rate) is the fraction of correct flips and FNR(False Negative rate) is the fraction of missed flips. These two metrics are called TFR (True Flip Rate) and MFR (Missed Flip Rate) respectively. There are also many others some of which are:

Demographic parity: $$P(\hat{y}\vert p) = P(\hat{y})$$

Disperate Impact Ratio (DIR): $$\frac{P(\hat{y}\vert p=minority)}{P(\hat{y}\vert p=majority)}$$

Disperate Impact Difference (DID): $$P(\hat{y}=1\vert p=majority) - P(\hat{y}=1\vert p=minority)$$

## FairLabel

Our algorithm consists of 2 sub algorithms - FairMin and FairMaj.

First of all, how FairMin works:
1. Split data into majority and minority groups
2. Train a classifier on the majority group
3. Get the classsifier's predictions for the minority group
4. Flip labels where the classifier has predicted 1 and the "true" label is 0

This way we make sure both classes are being treated equally.

Next we could apply FairMaj (optionally):
1. Split data into majority and majority groups
2. Train a classifier on the minority group
3. Get the classsifier's predictions for the minority group
4. Flip labels where the classifier has predicted 0 and the "true" label is 1

Now we are going to implement it using transformers and then combine it to an ultimate pipeline:

In [3]:
# Custom transformer for FAIRMIN
class FAIRMINTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.7, feature_index = 4):
        self.threshold = threshold
        self.feature_index = feature_index
        self.classifier_majority = LogisticRegression(random_state=42)

    def fit(self, X, y):
        indices_majority = np.where(X[self.feature_index] < self.threshold)[0]
        X_majority, y_majority = np.array([X[i] for i in indices_majority]), np.array([y[i] for i in indices_majority])
        self.classifier_majority.fit(X_majority, y_majority)
        return self

    def transform(self, X, y):
        X = self.X
        y = self.y
        indices_minority = np.where(X[self.feature_index] > self.threshold)[0]
        X_minority, y_minority = np.array([X[i] for i in indices_minority]), np.array([y[i] for i in indices_minority])
        predictions_minority = self.classifier_majority.predict(X_minority)
        flipped_labels_minority = np.where(predictions_minority > self.threshold, 1, 0)
        return np.concatenate((X, X_minority)), np.concatenate((y, flipped_labels_minority))

# Custom transformer for FAIRMAJ
class FAIRMAJTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.7, feature_index = 4):
        self.threshold = threshold
        self.feature_index = feature_index
        self.classifier_minority = LogisticRegression(random_state=42)

    def fit(self, X, y):
        indices_minority = np.where(X[self.feature_index] > self.threshold)[0]
        X_minority, y_minority = np.array([X[i] for i in indices_minority]), np.array([y[i] for i in indices_minority])
        self.classifier_minority.fit(X_minority, y_minority)
        return self

    def transform(self, X, y):
        indices_majority = np.where(X[self.feature_index] < self.threshold)[0]
        X_majority, y_majority = np.array([X[i] for i in indices_majority]), np.array([y[i] for i in indices_majority])
        predictions_majority = self.classifier_minority.predict(X_majority)
        flipped_labels_majority = np.where(predictions_majority > self.threshold, 1, 0)
        return np.concatenate((X_majority, X)), np.concatenate((flipped_labels_majority, y))

# Concatenate transformer
class ConcatenateTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return np.concatenate(X, axis=0), np.concatenate(y, axis=0)

In [4]:
# Pipeline
fairlabel_pipeline = Pipeline([
    ('fairmin', FAIRMINTransformer()),
    ('fairmaj', FAIRMAJTransformer()),
    ('concatenate', ConcatenateTransformer())
])

## Synthetic datasets for testing

To test our theory we will do three experiments with three different synthetic datasets:
* Linear
* Clusters around n-hypercubes
* Gaussian Quantiles

We will generate the linear dataset with the function below (it inserts noise also into the data, as it should):

In [5]:
def generate_linear_dataset(n_samples, n_features, p_noise, feature_bias_index=None, feature_bias_threshold=0.0):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    def generate_random_coefficients(n_features):
        return np.random.randn(n_features)

    def generate_random_x(n_features, n_samples):
        return np.random.randn(n_samples, n_features)

    def generate_random_binary(n_samples):
        return np.random.choice([0, 1], size=n_samples)
    
    n_samples_perfect = int(n_samples * (1 - p_noise))
    n_samples_noise = n_samples - n_samples_perfect

    w = generate_random_coefficients(n_features)
    
    X = generate_random_x(n_features, n_samples_perfect)
    probs = sigmoid(np.dot(X, w))
    y = np.array([1 if i > 0.5 else 0 for i in probs]).reshape(n_samples_perfect)

    X_noise = generate_random_x(n_features, n_samples_noise)
    y_noise = generate_random_binary(n_samples_noise)
    
    if feature_bias_index is not None:
        biased_indices = X_noise[:, feature_bias_index] > feature_bias_threshold
        y_noise[biased_indices] = np.round(np.random.rand(len(y_noise[biased_indices])))
        
    X_full = np.concatenate((X, X_noise))
    y_full = np.concatenate((y, y_noise))

    return X_full, y_full

For the other 2 we are going to use sklearn.datasets to generate the datasets but since the sklearn functions do not have parameters for introducing bias, we are going to have to inject it ourselves with postprocessing.

In [6]:
def introduce_feature_bias(X, y, feature_index, bias_threshold):
    
    biased_indices = X[:, feature_index] > bias_threshold
    y[biased_indices] = np.round(np.random.rand(len(y[biased_indices])))
    return X, y

def make_gauss_biased(n_samples, n_features, bias_level):
    X, y = make_gaussian_quantiles(n_samples=n_samples, n_features=n_features, n_classes=2, random_state=42)

    X_biased, y_biased = introduce_feature_bias(X, y, 4, bias_level)
    return X_biased, y_biased
    
def make_hypercubes_biased(n_samples, n_features, bias_level):
    X, y = make_classification(n_samples, n_features, random_state=42)

    X_biased, y_biased = introduce_feature_bias(X, y, 4, bias_level)
    return X_biased, y_biased

Now let's get our datasets ready, splitting them 80/20 for training and testing:

In [7]:
X_lin, y_lin = generate_linear_dataset(2000, 7, 0.5, feature_bias_index=4, feature_bias_threshold=0.7)
X_gauss, y_gauss = make_gauss_biased(2000, 7, 0.7)
X_hcube, y_hcube = make_hypercubes_biased(2000, 7, 0.7)

In [8]:
X_lin_train, X_lin_test, y_lin_train, y_lin_test = train_test_split(X_lin, y_lin, test_size=0.2, random_state=42)

## Experiments

Disclaimer: This is the furthest I got because there seems to be a mistake and I feel lost looking at the code above.

In [9]:
normal_lr = LogisticRegression()

In [10]:
pd.concat((pd.DataFrame(X_lin), pd.Series(y_lin)), axis = 1).corr()

,0,1,2,3,4,5,6,0
0,1.000000,-0.003091,-0.012820,-0.017452,0.012557,0.012079,0.016363,-0.083337
1,-0.003091,1.000000,0.013164,-0.043700,0.019759,-0.020006,-0.017279,-0.047186
2,-0.012820,0.013164,1.000000,-0.000100,-0.020588,0.026882,0.001786,0.032505
3,-0.017452,-0.043700,-0.000100,1.000000,0.007443,-0.018699,0.016272,-0.236125
4,0.012557,0.019759,-0.020588,0.007443,1.000000,0.031215,0.008602,-0.159333
5,0.012079,-0.020006,0.026882,-0.018699,0.031215,1.000000,0.034128,0.156498
6,0.016363,-0.017279,0.001786,0.016272,0.008602,0.034128,1.000000,-0.261428
0,-0.083337,-0.047186,0.032505,-0.236125,-0.159333,0.156498,-0.261428,1.000000


In [11]:
fl_lr = LogisticRegression()

In [12]:
test1, test2 = fairlabel_pipeline.fit(X_lin_train, y=y_lin_train)

TypeError: transform() missing 1 required positional argument: 'y'

In [ ]:
FAIRMINTransformer().fit_transform(X_lin_train, y_lin_train)